In [2]:
import pypyodbc as odbc #!pip install pypyodbc
import pandas as pd
import time
import os

print(odbc.drivers())

['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'ODBC Driver 17 for SQL Server', 'SQL Server Native Client RDA 11.0', 'SQL Server Native Client RDA 11.0']


#### Needed Inputs:

In [3]:
driverName = "SQL Server" 
serverName = "DESKTOP-UVMKETP\SQLEXPRESS"
dataBaseName = "SQL_PowerBI_Test"
username = "jvenncpe"
pwd = "#Juv1994*"
tableName = "olist_orders_dataset"

csv_FilePaTH = r"G:\00 Work Files\02 Work Trainings & Apps\00 Project Portfolio\Ecommerce_Amazon (Python, SQL, PowerBI)\CSV Files\Amazon_Olist"
csv_FolderName = r"olist_orders_dataset"
csv_FileName = r"olist_orders_dataset.csv" #To be used for column name reference in SQL Table

### Custom Functions

In [4]:
def updateSQL():
    try:
        conn.commit()
        return print("SQL Updated Successfully!")
        
    except Exception as e:
        return print(f"Error occurred: {e}")

def SQL_createTable_columnName(column):
    column_names = []
    for x in column:
        x = f'[{x}] VARCHAR(MAX)'
        column_names.append(x)
    return ', '.join(column_names)  # Join the column names with commas

def SQL_createTable_query(tableName,tableColumns):
    createTable_query = f"\
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{tableName}')\
    BEGIN\
        CREATE TABLE {dataBaseName}.dbo.{tableName}({tableColumns})\
    END".strip()
    return createTable_query

def bulk_insert(datapath_file, SQL_Table):
    insertBulk_query = f"""
        BULK INSERT {SQL_Table}
        FROM '{datapath_file}'
        WITH
        (
            FIELDQUOTE = '"',
            FIRSTROW = 2,
            FIELDTERMINATOR = ',', 
            ROWTERMINATOR = '\n',  
            TABLOCK
        ) 
    """.strip()
    return insertBulk_query

#### Adhoc SQL Configuration

In [5]:
connection_string = f"""
    DRIVER={{{driverName}}};
    SERVER={serverName};
    DATABASE={dataBaseName};
    Trust_Connection=yes;
    UID={username};
    PWD={pwd};
"""
try:
    conn = odbc.connect(connection_string)
    cursor = conn.cursor()
    print("Connection successful!")

except Exception as e:
    print(f"Error: {e}")


Connection successful!


#### Main Function

In [6]:
if __name__ == '__main__':
    csv_filePaTH_REF = os.path.join(csv_FilePaTH,csv_FolderName,csv_FileName)   
    csv_FoldderPath = os.path.join(csv_FilePaTH, csv_FolderName)
    
    df_productDetails = pd.read_csv(csv_filePaTH_REF, encoding='latin1', na_values=[""])
    
    for columnName in df_productDetails.columns:
        columnName_After = columnName.replace(".","")
        columnName_After = columnName_After.replace(" ","_")
        df_productDetails.rename(columns={columnName: columnName_After}, inplace=True)
    
    cursor = conn.cursor()
    tableColumns = SQL_createTable_columnName(df_productDetails.columns)
    
    try:
        with cursor:
            cursor.execute(SQL_createTable_query(tableName,tableColumns))
            updateSQL()
    except Exception as e:
        print(e)
        conn.rollback()
        print('Transaction rollback')
    
    data_files = os.listdir(csv_FoldderPath)
    SQL_Table = f'{dataBaseName}.dbo.{tableName}'
    
    cursor = conn.cursor()
    try:
        with cursor:
            print(f"Target SQL Table: {SQL_Table}")
            print(f"---------------------------")
            print(f"Files to Upload:")
            
            for data_file in data_files:
                if data_file.endswith('.csv'):
                    cursor.execute(bulk_insert(os.path.join(csv_FoldderPath, data_file), SQL_Table))
                    print(data_file)
                    
            print(f"---------------------------")
            updateSQL()
            
    except Exception as e:
        print(e)
        conn.rollback()
        print('Transaction rollback')

SQL Updated Successfully!
Target SQL Table: SQL_PowerBI_Test.dbo.olist_orders_dataset
---------------------------
Files to Upload:
olist_orders_dataset.csv
---------------------------
SQL Updated Successfully!


In [7]:
df_productDetails.head(20)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,02/10/2017 10:56,02/10/2017 11:07,04/10/2017 19:55,10/10/2017 21:25,18/10/2017 0:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,24/07/2018 20:41,26/07/2018 3:24,26/07/2018 14:31,07/08/2018 15:27,13/08/2018 0:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,08/08/2018 8:38,08/08/2018 8:55,08/08/2018 13:50,17/08/2018 18:06,04/09/2018 0:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,18/11/2017 19:28,18/11/2017 19:45,22/11/2017 13:39,02/12/2017 0:28,15/12/2017 0:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,13/02/2018 21:18,13/02/2018 22:20,14/02/2018 19:46,16/02/2018 18:17,26/02/2018 0:00
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,09/07/2017 21:57,09/07/2017 22:10,11/07/2017 14:58,26/07/2017 10:57,01/08/2017 0:00
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,11/04/2017 12:22,13/04/2017 13:25,NaN,NaN,09/05/2017 0:00
7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,16/05/2017 13:10,16/05/2017 13:22,22/05/2017 10:07,26/05/2017 12:55,07/06/2017 0:00
8,76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,23/01/2017 18:29,25/01/2017 2:50,26/01/2017 14:16,02/02/2017 14:08,06/03/2017 0:00
9,e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,29/07/2017 11:55,29/07/2017 12:05,10/08/2017 19:45,16/08/2017 17:14,23/08/2017 0:00
